In [2]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")
import random


# importing the nltk suite 
import nltk
  
# importing jaccard distance
# and ngrams from nltk.util
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.metrics import edit_distance


In [70]:

# Downloading and importing
# package 'words' from nltk corpus
nltk.download('words')
from nltk.corpus import words
  
  
correct_words = words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Manoj\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [72]:

# list of incorrect spellings
# that need to be corrected 
incorrect_words=['Adreenaline']
  
# loop for finding correct spellings
# based on jaccard distance
# and printing the correct word
for word in incorrect_words:
    temp = [(jaccard_distance(set(ngrams(word, 2)),
                              set(ngrams(w, 2))),w)
            for w in correct_words if w[0]==word[0]]
    print(sorted(temp, key = lambda val:val[0])[0][1])

Adrenalin


In [11]:
# pip install autocorrect

In [12]:
from autocorrect import Speller

spell = Speller(lang='en')


In [14]:
import spacy

In [15]:
train = pd.read_csv(r"C:\Users\Manoj\praxis Term 3\Cap Stone\capstone project\train.csv")
features = pd.read_csv(r"C:\Users\Manoj\praxis Term 3\Cap Stone\capstone project\features.csv")
patient_notes = pd.read_csv(r"C:\Users\Manoj\praxis Term 3\Cap Stone\capstone project\patient_notes.csv")

In [16]:
# check this
training = train.copy()

In [17]:
from spacy.tokens import DocBin
import spacy
import json
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import random

# For feature 4

In [68]:
new_train = training[(training.feature_num == 4)][['pn_num','annotation',"location"]]

In [69]:
new_train = new_train[new_train.annotation != '[]']

In [70]:
new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

In [71]:
new_train.loc[new_train.location == "'883 900'",'annotation'] = "['dad  heart attack']"

In [72]:
new_train = (new_train.set_index(['pn_num']) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))

In [73]:
new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])

In [74]:
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])

In [75]:
#new_train["annotation"] = new_train["annotation"].apply(lambda x: spell(x))

In [76]:
#intersecting = set(patient_notes.pn_num).intersection(new_train.pn_num)
#for i in intersecting:
#    for row_index,row in new_train[new_train.pn_num == i].iterrows():  # for each unique patient number 
#        new_train.loc[row_index,"patientnotes"] = patient_notes.loc[i,"pn_history"]
#new_train["patientnotes"] = new_train["patientnotes"].apply(lambda x: spell(x))

In [77]:
pn_Num_list = list(new_train.pn_num.unique())

In [78]:
len(pn_Num_list)

50

In [79]:
final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in new_train[new_train.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature")) # store the locations in numeric format
        
    text = patient_notes[patient_notes.pn_num == patient_number]["pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
            
            

In [80]:
len(final_train)

50

In [81]:
train = final_train[:40]
validation = final_train[40:]

In [82]:
# converting the json file to spacy file in order to use it in training model

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [83]:
#train data set

camp_train = create_training(train)
camp_train.to_disk("nbme_train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 365.09it/s]


In [84]:
#test data set

camp_train = create_training(validation)
camp_train.to_disk("nbme_validation.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 623.85it/s]


In [85]:
! python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-04-03 10:38:39.722937: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-03 10:38:39.722979: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [86]:
! python -m spacy train config.cfg --output ./output

[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     70.33    0.00    0.00    0.00    0.00
  5     200         23.91   1280.55   66.67   75.00   60.00    0.67
 10     400        794.48     77.28   70.00   70.00   70.00    0.70
 15     600          0.12      0.18   70.00   70.00   70.00    0.70
 20     800          0.00      0.00   70.00   70.00   70.00    0.70
 25    1000          0.00      0.00   70.00   70.00   70.00    0.70
 30    1200          0.00      0.00   70.00   70.00   70.00    0.70
 35    1400          0.00      0.00   63.16   66.67   60.00    0.63
 40    1600          

2022-04-03 10:38:56.468185: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-03 10:38:56.468234: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-04-03 10:38:59,139] [INFO] Set up nlp object from config
[2022-04-03 10:38:59,139] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-04-03 10:38:59,154] [INFO] Created vocabulary
[2022-04-03 10:38:59,154] [INFO] Finished initializing nlp object
[2022-04-03 10:38:59,495] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [41]:
! python -m spacy train config.cfg --output ./output

[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     70.33    0.00    0.00    0.00    0.00
  5     200         23.91   1280.55   66.67   75.00   60.00    0.67
 10     400        794.48     77.28   70.00   70.00   70.00    0.70
 15     600          0.12      0.18   70.00   70.00   70.00    0.70
 20     800          0.00      0.00   70.00   70.00   70.00    0.70
 25    1000          0.00      0.00   70.00   70.00   70.00    0.70
 30    1200          0.00      0.00   70.00   70.00   70.00    0.70
 35    1400          0.00      0.00   63.16   66.67   60.00    0.63
 40    1600          

2022-03-31 14:41:59.660188: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-31 14:41:59.660240: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-03-31 14:42:03,016] [INFO] Set up nlp object from config
[2022-03-31 14:42:03,028] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-31 14:42:03,031] [INFO] Created vocabulary
[2022-03-31 14:42:03,033] [INFO] Finished initializing nlp object
[2022-03-31 14:42:03,449] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


# For Feature 5

In [87]:
new_train = training[(training.feature_num == 5)][['pn_num','annotation',"location"]]

In [88]:
new_train.loc[new_train.location == "'883 900'",'annotation'] = "['dad  heart attack']"

In [89]:
new_train = new_train[new_train.annotation != '[]']

In [90]:
new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

In [91]:
new_train = (new_train.set_index(['pn_num']) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))

In [92]:
new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])

In [93]:
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])

In [94]:
new_train[new_train.start_location == "452"]

,pn_num,annotation,location,start_location,end_location
45,2198,'DENIES TEMPERATURE INTOLERANCE','452 458;478 501',452,501
46,2198,'DENIES SKIN/HAIR CHANGES','452 476',452,476


In [95]:
new_train[new_train.end_location == "501"]

,pn_num,annotation,location,start_location,end_location
45,2198,'DENIES TEMPERATURE INTOLERANCE','452 458;478 501',452,501


In [96]:
new_train = new_train.drop(index=46)

In [97]:
new_train = new_train.drop(index=3)

In [98]:
new_train = new_train.drop(index=6)

In [99]:
new_train = new_train.drop(index=9)

In [100]:
new_train = new_train.drop(index=19)

In [101]:
new_train = new_train.drop(index=25)

In [102]:
new_train = new_train.drop(index=29)

In [103]:
new_train = new_train.drop(index=22)

In [104]:
#new_train["annotation"] = new_train["annotation"].apply(lambda x: spell(x))

In [105]:
#intersecting = set(patient_notes.pn_num).intersection(new_train.pn_num)
#for i in intersecting:
#    for row_index,row in new_train[new_train.pn_num == i].iterrows():  # for each unique patient number 
#        new_train.loc[row_index,"patientnotes"] = patient_notes.loc[i,"pn_history"]
#new_train["patientnotes"] = new_train["patientnotes"].apply(lambda x: spell(x))

In [106]:
pn_Num_list = list(new_train.pn_num.unique())

In [107]:
len(pn_Num_list)

42

In [108]:
final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in new_train[new_train.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature")) # store the locations in numeric format
        
    text = patient_notes[patient_notes.pn_num == patient_number]["pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
            
            

In [109]:
len(final_train)

42

In [110]:
train = final_train[:33]
validation = final_train[33:]

In [111]:
# converting the json file to spacy file in order to use it in training model

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [112]:
#train data set

camp_train = create_training(train)
camp_train.to_disk("nbme_train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 510.74it/s]


In [113]:
#test data set

camp_train = create_training(validation)
camp_train.to_disk("nbme_validation.spacy")

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 579.77it/s]


In [114]:
! python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-04-03 10:46:01.729849: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-03 10:46:01.729926: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [68]:
! python -m spacy train config.cfg --output ./output

[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================


2022-03-31 14:48:56.517700: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-31 14:48:56.517773: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-03-31 14:49:06,523] [INFO] Set up nlp object from config
[2022-03-31 14:49:06,556] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-31 14:49:06,566] [INFO] Created vocabulary
[2022-03-31 14:49:06,568] [INFO] Finished initializing nlp object
[2022-03-31 14:49:07,753] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     69.33    0.00    0.00    0.00    0.00
  6     200       4035.48   2437.36   14.29   20.00   11.11    0.14
 12     400         31.24     75.87   44.44   44.44   44.44    0.44
 18     600         20.79     23.06   26.67   33.33   22.22    0.27
 24     800         23.13     24.17   28.57   25.00   33.33    0.29
 30    1000        280.00     33.58   34.78   28.57   44.44    0.35
 36    1200          2.38      1.87   33.33   33.33   33.33    0.33
 42    1400         12.22      4.01   42.11   40.00   44.44    0.42
 48    1600         85.82     26.59   25.00   28.57   22.22    0.25
 54    1800        546.44     22.56   22.22   22.22   22.22    0.22
 60    2000      